In [ ]:
import gym
import sys
sys.path.append('./src')

#env_id = "ALE/Pong-v5"
env_id = 'CartPole-v1'
# Create the env
env = gym.make(env_id)

env.reset()


In [ ]:
from feedback import FeedbackManager

#fb = FeedbackManager("notrealfile.dat", show_picker=False)
fb = FeedbackManager("testcartpole.dat", show_picker=False)

In [ ]:
import torch
device = torch.device("cuda")

In [ ]:
from rewardmodel import RewardModel

rewardmodel = RewardModel(env.observation_space.shape, device)

In [ ]:
for i in range(1000):
    batch = fb.randomBatch(20)
    rewardmodel.train(batch)

In [ ]:
env.reset()
state, _, done, _ = env.step(env.action_space.sample())
env.render()
if done:
    env.reset()
rewardmodel.evaluate(state)

In [ ]:
from stable_baselines3.common.env_checker import check_env

In [ ]:
from envwithrewardmodel import EnvWithRewardModel
def makeenvfun():
    def _f():
        base_env_id = 'CartPole-v1'
        base_env = gym.make(base_env_id)
        return EnvWithRewardModel(base_env, rewardmodel)
    return _f 

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv
vec_env = DummyVecEnv([makeenvfun() for i in range(4)])

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import A2C



model = A2C("MlpPolicy", vec_env, verbose=1)

In [ ]:
reward_target = 300
for i in range(1000):
    model.learn(total_timesteps=1000, reset_num_timesteps=False)

    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
    print(f"{(i+1) * 1000}: mean {mean_reward}, std {std_reward}")
    if mean_reward > reward_target:
        print(f"took {(i+1) * 1000} timesteps to reach mean reward {reward_target}")
        break;

In [ ]:
obs = env.reset()
total_reward = 0
for i in range(500):
    action, _states = model.predict(obs)
    obs, _, done, _ = env.step(action)
    reward = rewardmodel.evaluate(obs)
    print(reward)
    total_reward += reward
    if done:
        env.reset()
    env.render()
    
print(f"total reward: {total_reward}")